In [1]:
import os
import sys
# add /home/shiftpub/AMM-Python as a path
sys.path.append(os.path.join(os.getcwd(), '..'))
from env.new_amm import AMM
from env.market import MarketSimulator
from env.amm_env import DynamicAMM


2024-09-24 17:11:37.137968: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import deque
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

def run_simulation_for_sigma(set_sigma, num_runs=300):
    annualized_sigmas = []

    for _ in tqdm(range(num_runs), desc=f"Set Sigma: {set_sigma}", leave=False):
        market = MarketSimulator(sigma=set_sigma)
        amm = AMM(fee=0.0005)
        mkt_mid = deque(maxlen=1000)
        log_returns = deque(maxlen=999)

        sum_log_returns = 0
        sum_log_returns_sq = 0
        window_size = 999
        sigma_values = []

        for _ in range(market.steps):
            mkt_ask = market.get_ask_price('A')
            mkt_bid = market.get_bid_price('B')
            mid_price = (mkt_ask + mkt_bid) / 2

            if len(mkt_mid) > 0:
                last_price = mkt_mid[-1]
                log_return = np.log(mid_price / last_price)

                sum_log_returns += log_return
                sum_log_returns_sq += log_return ** 2

                log_returns.append(log_return)

                if len(log_returns) == window_size:
                    oldest_return = log_returns[0]
                    sum_log_returns -= oldest_return
                    sum_log_returns_sq -= oldest_return ** 2

            mkt_mid.append(mid_price)

            if len(mkt_mid) == 1000:
                mean_log_return = sum_log_returns / window_size
                variance = (sum_log_returns_sq / window_size) - (mean_log_return ** 2)
                current_sigma = np.sqrt(variance)
                sigma_values.append(current_sigma)

            market.next()

        sigma_mean = np.mean(sigma_values)
        annualized_sigma = sigma_mean * np.sqrt(252 * 6.5 * 60)
        annualized_sigmas.append(annualized_sigma)

    return set_sigma, annualized_sigmas

def collect_data_for_sigmas(sigma_values):
    boxplot_data = {}
    sigma_means = {}
    with ProcessPoolExecutor() as executor:
        futures = {executor.submit(run_simulation_for_sigma, sigma): sigma for sigma in sigma_values}

        for future in as_completed(futures):
            sigma, annualized_sigmas = future.result()
            boxplot_data[sigma] = annualized_sigmas
            sigma_means[sigma] = np.mean(annualized_sigmas)

    return boxplot_data, sigma_means

# Define the sigma values to set
sigma_values = np.round(np.arange(0.05, 0.36, 0.01), 3)

# Collect data for all sigma values using parallel processing
boxplot_data, sigma_means = collect_data_for_sigmas(sigma_values)

# Prepare data for the boxplot
data = [boxplot_data[sigma] for sigma in sigma_values]

# Plotting the boxplot
plt.figure(figsize=(15, 8))
plt.boxplot(data, labels=[f'{s:.2f}' for s in sigma_values])
plt.title("Boxplot of Annualized Sigma Values for Different Set Sigmas", fontsize=16)
plt.xlabel("Set Sigma", fontsize=14)
plt.ylabel("Annualized Sigma (Volatility)", fontsize=14)
plt.grid(True)
plt.savefig("sigma-boxplot.png")
plt.show()

# Prepare data for linear regression
set_sigma_array = np.array(list(sigma_means.keys())).reshape(-1, 1)
mean_annualized_sigma_array = np.array(list(sigma_means.values())).reshape(-1, 1)

# Fit the linear regression model
model = LinearRegression()
model.fit(set_sigma_array, mean_annualized_sigma_array)

# Extract the coefficient and intercept
coefficient = model.coef_[0][0]
intercept = model.intercept_[0]

# Predict the values and calculate R-squared
predicted_annualized_sigma = model.predict(set_sigma_array)
r_squared = model.score(set_sigma_array, mean_annualized_sigma_array)

# Plot the linear regression
plt.figure(figsize=(10, 6))
plt.scatter(set_sigma_array, mean_annualized_sigma_array, color='blue', label='Mean Annualized Sigma')
plt.plot(set_sigma_array, predicted_annualized_sigma, color='red', linewidth=2, label=f'Linear Fit (R² = {r_squared:.4f})\nCoef: {coefficient:.4f}, Intercept: {intercept:.4f}')
plt.title("Linear Regression of Set Sigma vs Mean Annualized Sigma", fontsize=16)
plt.xlabel("Set Sigma", fontsize=14)
plt.ylabel("Mean Annualized Sigma", fontsize=14)
plt.grid(True)
plt.legend()
plt.savefig("linear-regression-sigma.png")
plt.show()


In [2]:
amm = AMM()
market = MarketSimulator()
trader_dir = '/home/shiftpub/AMM-Python/models/trader_model'
env = DynamicAMM(amm=amm, market=market, trader_dir=trader_dir)

obs, _ = env.reset()
print(obs)

Loaded trader with competition level of 0.05
Loaded trader with competition level of 0.10
Loaded trader with competition level of 0.15
Loaded trader with competition level of 0.20
Loaded trader with competition level of 0.25
Loaded trader with competition level of 0.30
Loaded trader with competition level of 0.35
Loaded trader with competition level of 0.40
Loaded trader with competition level of 0.45
Loaded trader with competition level of 0.50
Loaded trader with competition level of 0.55
Loaded trader with competition level of 0.60
Loaded trader with competition level of 0.65
Loaded trader with competition level of 0.70
Loaded trader with competition level of 0.75
Loaded trader with competition level of 0.80
Loaded trader with competition level of 0.85
Loaded trader with competition level of 0.90
Loaded trader with competition level of 0.95
Loaded trader with competition level of 1.00
[1.0000004 0.9999996 1.        1.       ]


In [3]:
sample_action = env.action_space.sample()
print(sample_action)

[0.00343199]


In [6]:
obs, reward, done, truncated, info = env.step([sample_action])

print(f"obs: {obs}")

obs: [1.000294  1.0002931 1.        1.       ]


In [8]:
print(info['urgent_levels'])

{0.05: 0.32029974, 0.1: 0, 0.15: 0.08220658, 0.2: 0.2688409, 0.25: 0, 0.3: 0.61305875, 0.35: 0.45480883, 0.4: 0.5401744, 0.45: 0.17963561, 0.5: 0.20323929, 0.55: 0, 0.6: 0.52671796, 0.65: 0.10924062, 0.7: 0.5576987, 0.75: 0.15177527, 0.8: 0.3733947, 0.85: 0.11509696, 0.9: 0.62220275, 0.95: 0, 1.0: 0.0472129}


In [9]:
obs, reward, done, truncated, info = env.step([0.05])
print(info['urgent_levels'])

Trader 0.05 action: [0.00249389 0.31971526]
Trader 0.10 action: [0.00020638 0.00526792]
Trader 0.15 action: [0.00512702 0.09733626]
Trader 0.20 action: [-0.00233299  0.2965042 ]
Trader 0.25 action: [1.9751005e-03 2.6792288e-05]
Trader 0.30 action: [-0.00103524  0.50328255]
Trader 0.35 action: [0.00137854 0.45340443]
Trader 0.40 action: [0.00325134 0.5273311 ]
Trader 0.45 action: [-0.00168698  0.18670434]
Trader 0.50 action: [-0.00033108  0.22605631]
Trader 0.55 action: [-6.4969547e-03  4.9501657e-05]
Trader 0.60 action: [0.00357839 0.5379442 ]
Trader 0.65 action: [0.00195673 0.14446396]
Trader 0.70 action: [-0.00106313  0.5591819 ]
Trader 0.75 action: [0.00178633 0.13815251]
Trader 0.80 action: [0.00199657 0.30375642]
Trader 0.85 action: [0.00065156 0.1148144 ]
Trader 0.90 action: [0.00081716 0.62294513]
Trader 0.95 action: [-0.0007465   0.00279012]
Trader 1.00 action: [-0.00163219  0.05655926]
{0.05: 0.31971526, 0.1: 0.005267918, 0.15: 0.09733626, 0.2: 0.2965042, 0.25: 0, 0.3: 0.50328

In [10]:
urgent_levels = info['urgent_levels']
print(urgent_levels)    

{0.05: 0.31971526, 0.1: 0.005267918, 0.15: 0.09733626, 0.2: 0.2965042, 0.25: 0, 0.3: 0.50328255, 0.35: 0.45340443, 0.4: 0.5273311, 0.45: 0.18670434, 0.5: 0.22605631, 0.55: 0, 0.6: 0.5379442, 0.65: 0.14446396, 0.7: 0.5591819, 0.75: 0.13815251, 0.8: 0.30375642, 0.85: 0.1148144, 0.9: 0.62294513, 0.95: 0, 1.0: 0.056559265}


In [12]:
for mc, urgent_level in urgent_levels.items():
    print(mc, urgent_level)

0.05 0.31971526
0.1 0.005267918
0.15 0.09733626
0.2 0.2965042
0.25 0
0.3 0.50328255
0.35 0.45340443
0.4 0.5273311
0.45 0.18670434
0.5 0.22605631
0.55 0
0.6 0.5379442
0.65 0.14446396
0.7 0.5591819
0.75 0.13815251
0.8 0.30375642
0.85 0.1148144
0.9 0.62294513
0.95 0
1.0 0.056559265
